In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
import os
import cv2
from sklearn.model_selection import train_test_split
from tqdm import tqdm  # import tqdm for progress bar 

In [2]:
target_labels = ['Cherry_(including_sour)___healthy', 'Cherry_(including_sour)___Powdery_mildew', 'Strawberry___healthy', 'Strawberry___Leaf_scorch']

In [3]:
main_path = 'data_penyakit/dataset/train'

In [4]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [5]:
# Load the images and labels
X = []
y = []
for label in target_labels:
    label_dir = os.path.join(main_path, label)
    for img_file in os.listdir(label_dir):
        img_path = os.path.join(label_dir, img_file)
        img = load_img(img_path, target_size=(128, 128))
        img_array = img_to_array(img)
        X.append(img_array)
        y.append(target_labels.index(label))

In [6]:
from tensorflow.keras.utils import to_categorical

In [7]:
X = np.array(X)
y = to_categorical(y, num_classes=len(target_labels))

In [8]:
X.shape

(7107, 128, 128, 3)

In [9]:
y.shape

(7107, 4)

In [10]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
X_train.shape

(5685, 128, 128, 3)

In [12]:
X_test.shape

(1422, 128, 128, 3)

In [13]:
y_train.shape

(5685, 4)

In [14]:
y_test.shape

(1422, 4)

In [15]:
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.models import Sequential

In [16]:
len(target_labels)

4

In [17]:
cnn_model = Sequential()
cnn_model.add(Conv2D(32, kernel_size= (3,3), activation = 'relu',input_shape=(128,128,3)))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(64,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(64,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(96,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(32,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())

cnn_model.add(Dropout(0.2))
cnn_model.add(Flatten())
cnn_model.add(Dense(128, activation = 'relu'))
cnn_model.add(Dropout(0.3))
cnn_model.add(Dense(len(target_labels), activation='softmax'))
cnn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [18]:
# Train the CNN model
cnn_history = cnn_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10)

Epoch 1/10


2023-05-13 06:15:33.119787: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


178/178 [==============================] - 35s 192ms/step - loss: 0.1952 - accuracy: 0.9309 - val_loss: 0.2909 - val_accuracy: 0.9149
Epoch 2/10
178/178 [==============================] - 34s 192ms/step - loss: 0.0857 - accuracy: 0.9720 - val_loss: 1.7630 - val_accuracy: 0.6442
Epoch 3/10
178/178 [==============================] - 36s 203ms/step - loss: 0.0531 - accuracy: 0.9838 - val_loss: 0.3078 - val_accuracy: 0.9100
Epoch 4/10
178/178 [==============================] - 38s 214ms/step - loss: 0.0324 - accuracy: 0.9896 - val_loss: 0.0187 - val_accuracy: 0.9930
Epoch 5/10
178/178 [==============================] - 41s 230ms/step - loss: 0.0305 - accuracy: 0.9898 - val_loss: 2.6849 - val_accuracy: 0.5949
Epoch 6/10
178/178 [==============================] - 42s 236ms/step - loss: 0.0281 - accuracy: 0.9900 - val_loss: 0.1697 - val_accuracy: 0.9255
Epoch 7/10
178/178 [==============================] - 42s 238ms/step - loss: 0.0195 - accuracy: 0.9945 - val_loss: 0.1090 - val_accuracy: 0.9

In [19]:
# Evaluate the performance of the trained CNN model on the test set
cnn_scores = cnn_model.evaluate(X_test, y_test, verbose=0)
print("CNN Model Accuracy: %.2f%%" % (cnn_scores[1] * 100))

CNN Model Accuracy: 99.65%


In [20]:
cnn_scores

[0.01775125041604042, 0.9964838027954102]

In [21]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Use the model to predict the test dataset
y_pred_prob = cnn_model.predict(X_test)

# Convert probabilities to classes
y_pred = np.argmax(y_pred_prob, axis=1)

# If y_test is one-hot encoded, convert it to classes as well
y_test_classes = np.argmax(y_test, axis=1)

# Calculate accuracy
accuracy = accuracy_score(y_test_classes, y_pred)

# Calculate precision
precision = precision_score(y_test_classes, y_pred, average='weighted') 

# Calculate recall
recall = recall_score(y_test_classes, y_pred, average='weighted') 

# Calculate F1 score
f1 = f1_score(y_test_classes, y_pred, average='weighted')

# Print the metrics
print("Accuracy: %.2f%%" % (accuracy * 100))
print("Precision: %.2f%%" % (precision * 100))
print("Recall: %.2f%%" % (recall * 100))
print("F1 score: %.2f%%" % (f1 * 100))

45/45 [==============================] - 4s 80ms/step
Accuracy: 99.65%
Precision: 99.65%
Recall: 99.65%
F1 score: 99.65%


In [22]:
# Extract the features using the trained CNN model
cnn_features = cnn_model.predict(X_train)

178/178 [==============================] - 14s 77ms/step


In [23]:
from sklearn.ensemble import RandomForestClassifier

In [24]:
# Train a Random Forest classifier on the extracted features
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(cnn_features, np.argmax(y_train, axis=1))

RandomForestClassifier(random_state=42)

In [25]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Evaluate the performance of the Random Forest classifier on the test set
rf_features = cnn_model.predict(X_test)
rf_features = rf_features.reshape(rf_features.shape[0], -1)
rf_predictions = rf_classifier.predict(rf_features)
rf_accuracy = np.mean(rf_predictions == np.argmax(y_test, axis=1))
print("Random Forest Classifier Accuracy: %.2f%%" % (rf_accuracy * 100))

# Calculate and print the Precision
rf_precision = precision_score(np.argmax(y_test, axis=1), rf_predictions, average='weighted')
print("Random Forest Classifier Precision: %.2f" % rf_precision)

# Calculate and print the Recall
rf_recall = recall_score(np.argmax(y_test, axis=1), rf_predictions, average='weighted')
print("Random Forest Classifier Recall: %.2f" % rf_recall)

# Calculate and print the F1 score
rf_f1_score = f1_score(np.argmax(y_test, axis=1), rf_predictions, average='weighted')
print("Random Forest Classifier F1 Score: %.2f" % rf_f1_score)

45/45 [==============================] - 4s 82ms/step
Random Forest Classifier Accuracy: 99.86%
Random Forest Classifier Precision: 1.00
Random Forest Classifier Recall: 1.00
Random Forest Classifier F1 Score: 1.00
